In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
#import packages and formatting statements

import numpy as np
import math
from matplotlib import pyplot as plt
import astropy.io.fits as pyfits
from astropy.table import Table, join, Column
from astropy.wcs import WCS
from astropy.io import ascii
import os
from astropy.coordinates import SkyCoord, Angle
from astropy.nddata import Cutout2D
import astropy.units as u
from astropy.visualization.wcsaxes import SphericalCircle
from matplotlib.patches import Circle

import sys
sys.path.append('./Python')
from nearest import findNearest
from deprojectGalaxy import deproject

plt.rc('text',usetex=False)
fontsize=10
plt.rc('xtick',labelsize=fontsize)
plt.rc('ytick',labelsize=fontsize)

In [3]:
# Functions
def angDistToPc(x,galDist):
    return(galDist*10**6*np.tan(x*np.pi/180))

def nearestHII(galaxy, HII_file, centerCoord, pa, incl, galDist, SNra, SNdec):
    
    if os.path.isfile(HII_file):

        # read in fits files
        hdu_HII = pyfits.open(HII_file)
        HIIMap  = hdu_HII[0].data

        #Convert x & y pixels to ra and dec
        wcs      = WCS(hdu_HII[0].header, naxis=2)
        naxis    = wcs._naxis # size of image naxis[0] = x and [1] = y
        grid     = np.indices((naxis[1],naxis[0]))
        ra, dec  = wcs.wcs_pix2world(grid[1],grid[0],0)

        #deproject ra and dec to dx and dy
        radius, projang, dx, dy = deproject(center_coord=centerCoord, incl=incl, pa=pa, ra=ra, dec=dec, return_offset=True)

        #flatten data structures 
        f_HII  = HIIMap.flatten()
        f_ra   = ra.flatten()
        f_dec  = dec.flatten()    
        f_dx   = dx.flatten()
        f_dy   = dy.flatten()

        #remove nans
#         keep  = np.where(np.isfinite(f_HII))
        keep  = np.where(f_HII >= 0)
        map_ra    = f_ra[keep]
        map_dec   = f_dec[keep]
        map_HII   = f_HII[keep]
        map_dx    = f_dx[keep]
        map_dy    = f_dy[keep]

        SN_Rad, SN_PA, SN_dx, SN_dy = deproject(center_coord=centerCoord, incl=incl, pa=pa, ra=SNra, dec=SNdec, return_offset=True) 
        
        print("Here:", SN_Rad,SN_dx)
        ### Needs work! return idx to access the ra, dec ###
        nearestHII_ang, idx = findNearest(map_dx, SN_dx, map_dy, SN_dy)
        HIIra, HIIdec = map_ra[idx], map_dec[idx]
        
        nearestHII_pc = angDistToPc(nearestHII_ang,galDist)
        
        print("Nearest HII region [pc]", nearestHII_pc, galaxy)

    else:
        print("No file for ", galaxy)

        nearestHII_pc = float("nan")
        HIIra = float("nan")
        HIIdec = float("nan")
        
    return(nearestHII_pc, HIIra, HIIdec)


In [4]:
# Compile galaxy and OSC objects
# full catalog header: Name,Type,Host,RA hms,Dec dms,RA dds,Dec dds,DiscDate

GalaxyData = Table.read('../Data/0.MUSEData.csv', format='csv')
SampleData = Table.read('../Data/2.MUSESampleCat.csv', format='csv') 
SampleData.remove_column("Distance")

Data = join(GalaxyData, SampleData, keys = "Galaxy", join_type = 'inner')

MUSEres, MUSENatMaps, MUSEHII = Data["MUSERes"], Data["MUSENatMap"], Data["MUSEHII"]

Data[0:5]

Galaxy,Distance,GalRa,GalDec,PosAng,Incl,MUSENatRes,MUSEmapNat,MUSEmap150pc,MUSE_HII_reg,Supernova,Type,Ra,Dec,MUSERes,MUSENatMap,MUSE150pcMap,MUSEHII,InSample
str7,float64,float64,float64,float64,float64,float64,str71,str70,str50,str11,str12,float64,float64,float64,str71,str70,str50,str4
NGC1087,15.85,41.60492,-0.498717,359.1,42.9,0.92,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_Nat/NGC1087-0.92asec_MAPS.fits,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_150pc/NGC1087-150pc_MAPS.fits,../Data/HII_reg_masks/NGC1087_nebulae_mask_V2.fits,SN1995V,II,41.61152777777777,-0.4987861111111111,0.92,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_Nat/NGC1087-0.92asec_MAPS.fits,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_150pc/NGC1087-150pc_MAPS.fits,../Data/HII_reg_masks/NGC1087_nebulae_mask_V2.fits,True
NGC1300,18.99,49.920815,-19.411114,278.0,31.8,0.89,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_Nat/NGC1300-0.89asec_MAPS.fits,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_150pc/NGC1300-150pc_MAPS.fits,../Data/HII_reg_masks/NGC1300_nebulae_mask_V2.fits,SN2022acko,II,49.91245833333333,-19.39518888888889,0.89,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_Nat/NGC1300-0.89asec_MAPS.fits,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_150pc/NGC1300-150pc_MAPS.fits,../Data/HII_reg_masks/NGC1300_nebulae_mask_V2.fits,True
NGC1365,19.57,53.40152,-36.140404,201.1,55.4,1.15,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_Nat/NGC1365-1.15asec_MAPS.fits,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_150pc/NGC1365-150pc_MAPS.fits,../Data/HII_reg_masks/NGC1365_nebulae_mask_V2.fits,SN2012fr,Ia,53.40057916666666,-36.12676944444445,1.15,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_Nat/NGC1365-1.15asec_MAPS.fits,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_150pc/NGC1365-150pc_MAPS.fits,../Data/HII_reg_masks/NGC1365_nebulae_mask_V2.fits,True
NGC1365,19.57,53.40152,-36.140404,201.1,55.4,1.15,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_Nat/NGC1365-1.15asec_MAPS.fits,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_150pc/NGC1365-150pc_MAPS.fits,../Data/HII_reg_masks/NGC1365_nebulae_mask_V2.fits,SN2001du,II,53.3713125,-36.142111111111106,1.15,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_Nat/NGC1365-1.15asec_MAPS.fits,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_150pc/NGC1365-150pc_MAPS.fits,../Data/HII_reg_masks/NGC1365_nebulae_mask_V2.fits,True
NGC1365,19.57,53.40152,-36.140404,201.1,55.4,1.15,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_Nat/NGC1365-1.15asec_MAPS.fits,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_150pc/NGC1365-150pc_MAPS.fits,../Data/HII_reg_masks/NGC1365_nebulae_mask_V2.fits,SN1983V,Ic,53.381876388888884,-36.14859166666667,1.15,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_Nat/NGC1365-1.15asec_MAPS.fits,../../GalaxyFiles/MUSELargeFiles/MUSE2.2_150pc/NGC1365-150pc_MAPS.fits,../Data/HII_reg_masks/NGC1365_nebulae_mask_V2.fits,True


In [5]:
#group sample by galaxy
gal_groups = Data.group_by('Galaxy')
galaxies = gal_groups.groups.keys
indices = gal_groups.groups.indices
indices = indices.tolist()
indices.append(0)

gs, imgs, HIIimgs, names, types, ras, decs, dists, res = [],[],[],[],[],[],[],[],[]
galRa,galDec,galPA,galIncl = [],[],[],[]

for i in range(len(indices)-2):
    SNnames, SNtypes, SNras, SNdecs = [],[],[],[]
    gs.append(gal_groups["Galaxy"][indices[i]])
    imgs.append(gal_groups["MUSENatMap"][indices[i]])
    HIIimgs.append(gal_groups["MUSEHII"][indices[i]])
    dists.append(gal_groups["Distance"][indices[i]])
    res.append(gal_groups["MUSERes"][indices[i]])
    galRa.append(gal_groups["GalRa"][indices[i]])
    galDec.append(gal_groups["GalDec"][indices[i]])
    galPA.append(gal_groups["PosAng"][indices[i]])
    galIncl.append(gal_groups["Incl"][indices[i]])
    
    #if gal_groups["SN_name"][indices[i]] != "SN1989B" and gal_groups["SN_name"][indices[i]] != "SN1973R":
    # Attach first SN per galaxy to a list
    SNnames.append(gal_groups["Supernova"][indices[i]])
    SNtypes.append(gal_groups["Type"][indices[i]])
    SNras.append(gal_groups["Ra"][indices[i]])
    SNdecs.append(gal_groups["Dec"][indices[i]])
    j = 1
    # attach additional  SN to list when galaxy has more than one
    while(indices[i] + j < indices[i+1]):    
        #if gal_groups["SN_name"][indices[i]+j] != "SN1989B" and gal_groups["SN_name"][indices[i]+j] != "SN1973R":
        SNnames.append(gal_groups["Supernova"][indices[i]+j])
        SNtypes.append(gal_groups["Type"][indices[i]+j])
        SNras.append(gal_groups["Ra"][indices[i]+j])
        SNdecs.append(gal_groups["Dec"][indices[i]+j])
        j += 1
    names.append(SNnames)
    types.append(SNtypes)
    ras.append(SNras)
    decs.append(SNdecs)

print(gs)
print(ras)

['NGC1087', 'NGC1300', 'NGC1365', 'NGC1433', 'NGC1566', 'NGC1672', 'NGC3627', 'NGC4254', 'NGC4303', 'NGC4321']
[[41.61152777777777], [49.91245833333333], [53.40057916666666, 53.3713125, 53.381876388888884, 53.38349999999999], [55.52640625], [64.9725, 64.99512499999999, 65.0058875], [71.423975, 71.45607916666667], [170.04809375, 170.05796666666666, 170.07957933333336, 170.07072916666667, 170.0592513888889], [184.71836111111114, 184.71066666666667, 184.71690625, 184.70291666666668], [185.48760972222217, 185.47536458333335, 185.50184583333336, 185.4698125, 185.46016458333332, 185.48041875, 185.48991666666666], [185.72488194444443, 185.74543194444445, 185.6970833333333, 185.72892708333336, 185.73393055555553]]


In [6]:
#get things ready for plotting
centerCoords = [(galRa[i], galDec[i]) for i in range(len(gs))] 

for i in range(len(gs)):
    if galIncl[i] == 90.0:
        galIncl[i] = 80.0

def getPlotStuff(gal, img, res, HIIimg, names, types, ras, decs, dist, centerCoord, galPA, galIncl):
    plots, wcss, HIImaps, hdus, SNnames, SNtypes, SNras, SNdecs, SNgals = [],[],[],[],[],[],[],[],[]
    dists, resolution, inHII, nearHII, HIIra, HIIdec = [],[],[],[],[],[]

    for i in range(len(gal)):
        for j in range(len(names[i])):
            
            if os.path.isfile(img[i]) and os.path.isfile(HIIimg[i]):

                #get hdu & wcs for MUSE map
                ha_hdu = pyfits.open(img[i])
                hdus.append(ha_hdu)
                ha_map = ha_hdu["HA6562_FLUX"].data
                w = WCS(ha_hdu["HA6562_FLUX"].header, naxis=2)
                wcss.append(w)                
                plots.append(ha_map)
                resolution.append(res[i])
                
                # open HII map and find nearest HII region
                hii_hdu = pyfits.open(HIIimg[i])
                hii_map = hii_hdu[0].data
                HIImaps.append(hii_map)
                
                ### something broken here ###
                near, idx = nearestHII(gal[i], HIIimg[i], centerCoord[i], galPA[i], galIncl[i], dist[i], ras[i][j], decs[i][j])
                
                nearHII.append(near)
                HIIra.append(ra)
                HIIdec.append(dec)

                # organize SN information
                SNnames.append(names[i][j])
                SNtypes.append(types[i][j])
                SNras.append(ras[i][j]) 
                SNdecs.append(decs[i][j])
                SNgals.append(gal[i])
                dists.append(dist[i])
                
                #Check if SN is in HII region
                xval, yval = w.wcs_world2pix(ras[i][j],decs[i][j], 0)
                xVal = int(xval)
                yVal = int(yval)
                HIIval = hii_map[yVal, xVal]

                if HIIval == -1:
                    inHII.append("No ")
                else:
                    inHII.append("Yes ")

            else:
                #print("no file for ")
                pass

    return(plots, wcss, HIImaps, hdus, SNnames, SNtypes, SNras, SNdecs, SNgals, dists, resolution, inHII, nearestHII, HIIra, HIIdec)


In [7]:
# set up plots, wcs information, and HII maps for contours.
print(ras)
plots, wcss, HIImaps, hdus, SNnames, SNtypes, SNras, SNdecs, SNgals, dists, resolution, inHII, nearestHII, HIIra, HIIdec = getPlotStuff(gs, imgs, res, HIIimgs, names, types, ras, decs, dists, centerCoords, galPA, galIncl)

# print(SNnames)
print(len(SNnames))
print(len(inHII))
print(nearHII)
print(HIIra, HIIdec)




[[41.61152777777777], [49.91245833333333], [53.40057916666666, 53.3713125, 53.381876388888884, 53.38349999999999], [55.52640625], [64.9725, 64.99512499999999, 65.0058875], [71.423975, 71.45607916666667], [170.04809375, 170.05796666666666, 170.07957933333336, 170.07072916666667, 170.0592513888889], [184.71836111111114, 184.71066666666667, 184.71690625, 184.70291666666668], [185.48760972222217, 185.47536458333335, 185.50184583333336, 185.4698125, 185.46016458333332, 185.48041875, 185.48991666666666], [185.72488194444443, 185.74543194444445, 185.6970833333333, 185.72892708333336, 185.73393055555553]]
Here: 0.009019049623723309 -0.0001728891155591486
[1.73704558e-05] [1.73704558e-05]
Nearest HII region [pc] [4.80527061] NGC1087


ValueError: too many values to unpack (expected 2)

In [ ]:
!HERE

In [ ]:
#set up labels for legends
legendLabels = []

for i in range(len(SNnames)):
    legendlabel = []
    if SNtypes[i][0] == 'unclassified':
        legendlabel.append(SNnames[i] + ' '  + SNtypes[i]) 
    else:        
        legendlabel.append(SNnames[i] + ' Type '  + SNtypes[i])
    legendLabels.append(legendlabel)

# print(legendLabels)


In [ ]:
print(str(SNgals[10]))


In [ ]:
# set size of boxes

# Next draw out the spheres of influence
def findAngSize(sphere, dist):
    """ Finds the angular size in decimal degrees when given the 
        size of the sphere in pc and the
        distance to the galaxy in Mpc
    """
    
    angSizeRad = np.arctan(sphere/(dist*10**6))
    angSize = angSizeRad * (180/np.pi)
        
    return(angSize)

sphereSizes = [10,50,200,500]
angSizes = []

for i in range(len(dists)):
    
    ang10 = findAngSize(sphereSizes[0], dists[i])
    ang50 = findAngSize(sphereSizes[1], dists[i])
    ang200 = findAngSize(sphereSizes[2], dists[i])
    ang500 = findAngSize(sphereSizes[3], dists[i])
    
    angSizes.append([ang10, ang50, ang200, ang500])
    

In [ ]:
current_cmap = plt.get_cmap("magma")
current_cmap.set_bad("black")

NUM_SNE= 33
# halfBox = 0.00416667 # 15'' in degrees

m,n,p =0,0,0 # plot counters for total number, rows, columns
PLOTS_PER_ROW = 3

fig2, axs = plt.subplots(math.ceil(NUM_SNE/PLOTS_PER_ROW),PLOTS_PER_ROW, figsize=(1, 1))
fig = plt.figure(figsize = (10,50))

for j in range(len(SNgals)): # counter to count galaxy maps
    
    #set up legend label with type classification
    if SNtypes[j] != "Unclassified":
        leglab = "Type " + SNtypes[j] + ": " + inHII[j]
    else:
        leglab= SNtypes[j] + ": " + inHII[j]
    legStr = str(resolution[j]) + " asec"
    
    #set up data to use astropy's cutout2D to plot subsections of galaxy map
    data = plots[j]
    logData = np.log10(data)    
    HIIData = HIImaps[j]
    wcs=wcss[j]

    halfBox = angSizes[j][3]/2. # 500 pc in decimal degrees/2?
    #cutout2D needs skycoord position to carry units
    ra, dec = Angle(SNras[j] * u.degree), Angle(SNdecs[j] * u.degree)
    raRad, decRad  = ra.radian * u.rad, dec.radian * u.rad    
    position = SkyCoord(raRad, decRad) #position is center, use ra & dec of SN location
    #size = u.Quantity((20,20), u.arcsec) #size is size of box in arcsec 
    size = u.Quantity((angSizes[j][3],angSizes[j][3]), u.degree) #size is size of box in arcsec 
    # make 2D cutout, will assign a new wcs to cutout to keep track of coords
    cutout = Cutout2D(logData, position, size, wcs) 
    # use this new wcs when converting to pixels to add additional details to plot
    xval, yval = cutout.wcs.wcs_world2pix(ra,dec, 0)
    # get HII data for contours
    HIICutout = Cutout2D(HIIData, position, size, wcs)

    titleStr = SNnames[j].upper() + " (" + SNgals[j][0].upper() + ")"
    # make plot
    axs[m][n] = fig.add_subplot(11,3,p+1,projection = cutout.wcs)
    axs[m][n].set_title(titleStr, fontsize=14)            
    axs[m][n].imshow(cutout.data, cmap=current_cmap, aspect="equal", origin = "lower", vmin=1, vmax=4,interpolation = "nearest", zorder = 0)
    axs[m][n].scatter(xval, yval, color = "black", marker ="o", s = 60, zorder=2)
    axs[m][n].scatter(xval, yval, color = "white", marker = "o", s = 80, edgecolor="black",zorder = 2, label=leglab)
    axs[m][n].set_ylabel(" ")
    axs[m][n].set_xlabel(" ")
    axs[m][n].tick_params(axis = "both", direction = "in", length = 0)
    axs[m][n].contour(HIICutout.data, [3], colors = ["greenyellow"], linestyles = ("-"), zorder = 1)
   
    # add in second legend to say resolution
    resLeg = axs[m][n].legend([legStr],handlelength=0, handletextpad=0,loc="upper right", facecolor='white', framealpha=1.0)
    for item in resLeg.legendHandles:
        item.set_visible(False)
    axs[m][n].add_artist(resLeg)

    leg = axs[m][n].legend(handlelength=0, handletextpad=0, loc="lower left", facecolor='white', framealpha=1.0)
    for item in leg.legendHandles:
        item.set_visible(False)



    #get spheres of influence
#     spheres = angSizes[j][:-1]


#     #get Pixel distance for scalebar
#     pixDists=[]
#     for i in range(len(spheres)-1):
#         sphereEdgeRa, sphereEdgeDec = ra + Angle(spheres[i]*u.degree), dec + Angle(spheres[i]*u.degree)
#         sphereEdgex, sphereEdgey = cutout.wcs.wcs_world2pix(sphereEdgeRa, sphereEdgeDec, 0)
#         pixDistx, pixDisty = sphereEdgex-xval, sphereEdgey-yval
#         pixDists.append(np.sqrt((pixDistx**2 + pixDisty**2)/2))

#     lowerleftra  = ra.value + halfBox 
#     lowerleftdec = dec.value - halfBox + halfBox*0.1
#     val=spheres[1]

#     xArr, yArr = np.linspace(lowerleftra, lowerleftra-val, 100),np.linspace(lowerleftdec, lowerleftdec, 100)


    n+=1
    if n%PLOTS_PER_ROW==0:
        m+=1
        n=0
    p +=1

    plt.subplots_adjust(left = 0.1, bottom = 0.1, right = 0.99, top = 0.9, wspace = 0.3, hspace = 0.2)
#plt.tight_layout(pad=0.5, w_pad=6.8, h_pad=1.0)
fig.savefig("../Figures/Zooms_500pc.pdf", dpi=300)
plt.show()


In [ ]:
print(inHII, SNtypes)

In [ ]:
numInHII = sum('Yes' in s for s in inHII)


In [ ]:
IIinHII, IainHII, IbcinHII, UncInHII = 0,0,0,0

for i in range(len(inHII)):
    if "Yes" in inHII[i] and "Ia" in SNtypes[i]:
        IainHII += 1
    elif "Yes" in inHII[i] and "II" in SNtypes[i]:
        IIinHII += 1
    elif "Yes" in inHII[i] and ("Ib" in SNtypes[i] or "Ic" in SNtypes[i]):
        IbcinHII += 1
    elif "Yes" in inHII[i]:
        UncInHII += 1
        
print(IIinHII, IainHII, IbcinHII, UncInHII)

In [ ]:
print(len(SNtypes))

In [ ]:
numII = sum('II' in s for s in SNtypes)
numIa = sum('Ia' in s for s in SNtypes)
numIb = sum('Ib' in s for s in SNtypes)
numIc = sum('Ic' in s for s in SNtypes)

In [ ]:
print(numII, numIa, numIb, numIc)